In [1]:
import torch
import torch.nn as nn
import numpy as np

import datasets.datasets as dtset
import utils.losses as losses

from utils.weakener import Weakener
from models.model import MLP

from utils.trainig_testing import train_and_evaluate_gradients, train_and_evaluate

In [4]:
import pickle
f = open("Experimental_results(0.2)/Dataset.pkl","rb")
Data,Weak = pickle.load(f)
f.close()


In [3]:
Weak.virtual_labels(p=None, optimize = True, convex = False) #This is to create Virtual Labels
Data.include_weak(Weak.z)
Data.include_virtual(Weak.v)
trainloader,testloader = Data.get_dataloader(weak_labels='weak')

C:\Users\danibacaicoa\AppData\Roaming\Python\Python311\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


In [5]:
torch.bincount(Weak.z).shape
print(Weak.Z.shape[0])
#weights = torch.linspace(0, 1, steps=5)

1023


In [6]:
Weak.z.bincount(minlength=Weak.Z.shape[0])

tensor([794, 723, 367,  ...,   0,   0,   0])

In [7]:
Weak.M

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.34217728e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.34217728e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.35544321e-02, 3.35544321e-02],
       ...,
       [2.04800000e-06, 2.04800000e-06, 2.04800000e-06, ...,
        2.04800000e-06, 0.00000000e+00, 2.04800000e-06],
       [2.04800000e-06, 2.04800000e-06, 2.04800000e-06, ...,
        2.04800000e-06, 2.04800000e-06, 0.00000000e+00],
       [5.11999944e-07, 5.11999944e-07, 5.11999944e-07, ...,
        5.11999944e-07, 5.11999944e-07, 5.11999944e-07]])

In [9]:
loss = losses.ForwardLoss(Weak.M)
overall_results = {}
overall_models = {}
epochs = 5
for i in range(2):
    mlp = MLP(Data.num_features,[Data.num_features],Data.num_classes, dropout_p=0.5, bn=True, activation =  'gelu')
    optim = torch.optim.Adam(mlp.parameters(),lr=1e-2)
    mlp, results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss,num_epochs=epochs,sound=1)
    overall_results[i] = results
    overall_models[i] = mlp  

Epoch 1/5: Train Loss: 94.6885, Train Acc: 0.1033, Test Acc: 0.0891
Epoch 2/5: Train Loss: 94.2416, Train Acc: 0.1017, Test Acc: 0.1032
Epoch 3/5: Train Loss: 94.2474, Train Acc: 0.1033, Test Acc: 0.1088
Epoch 4/5: Train Loss: 94.2538, Train Acc: 0.1022, Test Acc: 0.0943


KeyboardInterrupt: 